In [1]:
from ase import units

from ase.lattice.cubic import FaceCenteredCubic
from ase.visualize import  view 
from ase.io import read

from ase.io.trajectory import Trajectory
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.nptberendsen import  NPTBerendsen
from ase.calculators.emt import  EMT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.optimize import  BFGS, LBFGS, GPMin, FIRE, FIRE2

In [2]:
size = 3

In [3]:
cu_struct = FaceCenteredCubic(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    symbol='Cu',
    size=(size, size, size),
    pbc=True,
)

view(cu_struct)

<Popen: returncode: None args: ['c:\\Users\\Acer\\anaconda3\\python.exe', '-...>

In [4]:
cu_struct.calc  = EMT()

In [5]:
cu_struct.get_potential_energy()

-0.6136032267264113

In [6]:
# dyn = BFGS(atoms=cu_struct).run(fmax=0.005)

In [6]:
MaxwellBoltzmannDistribution(atoms=cu_struct, temperature_K=400)

In [8]:
# cu_struct.get_momenta()

### NVE

In [7]:
velet =  VelocityVerlet(cu_struct, 5 * units.fs, trajectory='md.traj', logfile='md.log')


velet.run(500)

True

In [11]:
def printenergy(a=cu_struct):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print(
        'Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
        'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin)
    )

In [8]:
read_struct  = read('md.traj', index=':')

view(read_struct)

<Popen: returncode: None args: ['c:\\Users\\Acer\\anaconda3\\python.exe', '-...>

### NVT

###### Langevin

In [9]:
dyn = Langevin(
    cu_struct,
    timestep=5.0 * units.fs,
    temperature_K=400.0,  # temperature in K
    friction=0.01 / units.fs,
)

In [12]:
dyn.attach(printenergy, interval=10)

traj = Trajectory('cu_langevin.traj', 'w', cu_struct)
dyn.attach(traj.write, interval=50)

# Now run the dynamics
printenergy()
dyn.run(500)

Energy per atom: Epot = 0.020eV  Ekin = 0.027eV (T=212K)  Etot = 0.047eV
Energy per atom: Epot = 0.020eV  Ekin = 0.027eV (T=212K)  Etot = 0.047eV
Energy per atom: Epot = 0.028eV  Ekin = 0.037eV (T=285K)  Etot = 0.065eV
Energy per atom: Epot = 0.035eV  Ekin = 0.042eV (T=327K)  Etot = 0.077eV
Energy per atom: Epot = 0.040eV  Ekin = 0.047eV (T=366K)  Etot = 0.087eV
Energy per atom: Epot = 0.046eV  Ekin = 0.047eV (T=360K)  Etot = 0.092eV
Energy per atom: Epot = 0.049eV  Ekin = 0.055eV (T=429K)  Etot = 0.104eV
Energy per atom: Epot = 0.044eV  Ekin = 0.059eV (T=454K)  Etot = 0.103eV
Energy per atom: Epot = 0.053eV  Ekin = 0.053eV (T=409K)  Etot = 0.106eV
Energy per atom: Epot = 0.056eV  Ekin = 0.050eV (T=384K)  Etot = 0.106eV
Energy per atom: Epot = 0.045eV  Ekin = 0.059eV (T=456K)  Etot = 0.104eV
Energy per atom: Epot = 0.051eV  Ekin = 0.054eV (T=417K)  Etot = 0.105eV
Energy per atom: Epot = 0.052eV  Ekin = 0.051eV (T=397K)  Etot = 0.104eV
Energy per atom: Epot = 0.049eV  Ekin = 0.060eV (T=

True

In [13]:
read_struct  = read('cu_langevin.traj', index=':')

view(read_struct)

<Popen: returncode: None args: ['c:\\Users\\Acer\\anaconda3\\python.exe', '-...>

### NPT

###### npt berendsen

In [14]:
dyn = NPTBerendsen(cu_struct, timestep=0.1 * units.fs, temperature_K=300,
                   taut=100 * units.fs, pressure_au=1.01325 * units.bar,
                   taup=1000 * units.fs, compressibility_au=4.57e-5 / units.bar, trajectory ='cu_berendsen.traj' )

In [15]:
dyn.run(500)

True

In [16]:
read_struct  = read('cu_berendsen.traj', index=':')

view(read_struct)

<Popen: returncode: None args: ['c:\\Users\\Acer\\anaconda3\\python.exe', '-...>